In [15]:
from dataclasses import dataclass
import torch
from typing import List, Union, Dict
import numpy as np
from datasets import load_from_disk
from transformers import AutoModel, TrainingArguments, Trainer
from pathlib import Path

In [4]:
def _make_mask(padded_batch, pad_token=0):
    return (padded_batch != pad_token).to(torch.uint8)


def _pad_truncate_add_special_tokens(batch: List[List[int]], max_len, pad_token=0, start_token=101, sep_token=102):
    sequence_lengths = torch.tensor([min(max_len-2, len(seq)) for seq in batch], dtype=torch.int64)
    batch_max_len = sequence_lengths.max()
    padded_batch = torch.full(size=(len(batch), batch_max_len+2), fill_value=pad_token, dtype=torch.int64)
    padded_batch[:, 0] = start_token
    for seq_idx, seq in enumerate(batch):
        padded_batch[seq_idx, 1:sequence_lengths[seq_idx]+1] = torch.tensor(seq[:sequence_lengths[seq_idx]], dtype=torch.int64)
        padded_batch[seq_idx, sequence_lengths[seq_idx]+1] = sep_token
    mask = _make_mask(padded_batch)
    return padded_batch, mask


@dataclass
class DataCollatorForPreprocessedICT:
    target_max_seq:int = 512
    context_max_seq:int = 512
    start_token:int = 101 # [CLS]
    sep_token:int = 102 # [SEP]
    pad_token:int = 0
        
    def _pad_truncate_add_special_tokens(self, batch: List[List[int]], max_len):
        return _pad_truncate_add_special_tokens(batch, max_len=max_len, pad_token=self.pad_token, start_token=self.start_token, sep_token=self.sep_token)
             
    def __call__(self, contexts: List[Dict[str, List[int]]]):
        if isinstance(contexts[0], dict):
            target_sentences = [context_dict['target'] for context_dict in contexts]
            flattened_contexts = [context_dict['flat_context'] for context_dict in contexts]
        correct_class = torch.arange(len(target_sentences), dtype=torch.int64)
        padded_target_batch, padded_target_mask = self._pad_truncate_add_special_tokens(target_sentences, self.target_max_seq)
        padded_context_batch, padded_context_mask = self._pad_truncate_add_special_tokens(flattened_contexts, self.context_max_seq)
        return {'target': padded_target_batch,
                'target_mask': padded_target_mask,
                'context': padded_context_batch,
                'context_mask': padded_context_mask,
                'correct_class': correct_class}

In [17]:
data_collator = DataCollatorForPreprocessedICT()

In [7]:
eval_dataset = load_from_disk('/home/cernypro/dev/source/ml4logs/data/processed/Tr-1093568_Ev-6400_Epochs-3_Seed-13/eval')

In [9]:
pretrained_model_name = "distilbert-base-cased"

In [10]:
class ClsEncoderTower(torch.nn.Module):
    """
    Simple model on top of a BERT like model.
    It's a linear layer on the [CLS] token of each sentence from BERT.
    """
    def __init__(self, pretrained_model_name_or_path, output_encode_dimension=512):
        super(ClsEncoderTower, self).__init__()
        self.bert = AutoModel.from_pretrained(pretrained_model_name_or_path)
        self.linear = torch.nn.Linear(self.bert.config.dim, output_encode_dimension) # self.bert.config.dim most likely 768
        
    def forward(self, input_ids, attention_mask):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_token_embedding = bert_output[0][:, 0]
        cls_encoding = self.linear(cls_token_embedding)
        return cls_encoding
    
class OneTowerICT(torch.nn.Module):
    """
    Network for the inverse close task, uses one BERT tower for creating encodings of target and context sentences (query and document as per nomenclature of original paper)
    Uses cross entropy loss
    """
    def __init__(self, pretrained_model_name_or_path, output_encode_dimension=512):
        super(OneTowerICT, self).__init__()
        self.tower = ClsEncoderTower(pretrained_model_name_or_path, output_encode_dimension)
        self.loss_fn = torch.nn.CrossEntropyLoss()
    def forward(self, target, target_mask, context, context_mask, correct_class):
        target_cls_encode = self.tower(input_ids=target, attention_mask=target_mask)
        context_cls_encode = self.tower(input_ids=context, attention_mask=context_mask)
        
        logits = torch.matmul(target_cls_encode, context_cls_encode.transpose(-2, -1))
        loss = self.loss_fn(logits, correct_class)
        return loss, target_cls_encode, context_cls_encode

In [12]:
saved_model_dir = Path.cwd().parent / 'models' / '1T_Eps_2_Lines_8000000_T-len_512_C-len_512_Tr-batch_64_Ev-b_64_O-dim_512'
saved_model_file = saved_model_dir / 'pytorch_model.bin'

In [13]:
state_dict = torch.load(saved_model_file)

In [20]:
model = OneTowerICT(pretrained_model_name)
model.load_state_dict(state_dict)
model.to('cuda')

OneTowerICT(
  (tower): ClsEncoderTower(
    (bert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(28996, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (ffn): FFN(
              (dropout):

In [21]:
training_args = TrainingArguments(output_dir=f"./tmpModelDir",
                                      per_device_eval_batch_size=64, 
                                      per_device_train_batch_size=64,
                                      warmup_steps=10,                # number of warmup steps for learning rate scheduler
                                      weight_decay=0.01,               # strength of weight decay
                                      logging_dir='../logs',            # directory for storing logs
                                      logging_steps=10,
                                      logging_first_step=True,
                                      eval_steps=20,
                                      evaluation_strategy='steps',
                                      prediction_loss_only=True,
                                      save_steps=100,
                                      save_total_limit=15,
                                      label_names=['target', 'context'],
                                      seed=42,
                                      report_to=[],
                                      remove_unused_columns=False)

In [22]:
trainer = Trainer(model=model,
                      args=training_args,
                      data_collator=data_collator,
                      eval_dataset=eval_dataset
                      )

In [23]:
trainer.evaluate()

{'eval_loss': 0.6826252937316895,
 'eval_runtime': 35.2668,
 'eval_samples_per_second': 181.474}